In [1]:
!pip install tensorflow

In [4]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

# Getting the file that we are going to do the training with
# We are going to use TensorFlow to download it
filepath = tf.keras.utils.get_file("shakespeare.txt", 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

# Converting the binary data into a human-readable Unicode
text = text[300000:800000]

# Creates a sorted list of unique characters from the text
characters = sorted(set(text))

# Creates dictionaries to map characters to indices and indices to characters
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

# Sets the sequence length and step size for creating training data
SEQ_LENGTH = 40
STEP_SIZE = 3

# Creates training data by generating sequences of length SEQ_LENGTH with a step size of STEP_SIZE
sentences = []
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i:i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH])

# Convert the training data to numpy
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        # Converts the training data into numpy arrays suitable for training a neural network.
        # x is a 3D array representing one-hot encoded input sequences.
        # y is a 2D array representing one-hot encoded target characters.
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1

# Building the neural network
model = Sequential()
# The input shape should be a tuple (SEQ_LENGTH, len(characters))
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))

model.add(Dense(len(characters)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
model.fit(x, y, batch_size=256, epochs=4)
model.save('textgenerator.model')


Epoch 1/4
651/651 [==============================] - 82s 121ms/step - loss: 2.7312
Epoch 2/4
651/651 [==============================] - 79s 122ms/step - loss: 2.3364
Epoch 3/4
651/651 [==============================] - 80s 122ms/step - loss: 2.2256
Epoch 4/4
651/651 [==============================] - 80s 123ms/step - loss: 2.1518


In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index:start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))  # Fix the shape of the array
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1  # Fix the indexing of the array
        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)  # Fix the typo in 'predictions'
        next_character = index_to_char[next_index]
        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

print('-----0.2-----')
print(generate_text(300, 0.2))


-----0.2-----
s a sea, for it is now the
sky: betwixt the sore the here the store the here the beart the sore the sore the will the sare the will stere the sear and the sere the seare the with the sher the wist the seard the sear the shat the seath the rore the wist in the the sore the shere the sore the the the sore the here the fore the shen the the 
